In [1]:
# Import Libraries
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import tensorflow as tf
from tqdm import tqdm_notebook
from scipy.spatial.distance import hamming, cosine
%matplotlib inline

/Users/steveyyp/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/steveyyp/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/steveyyp/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/steveyyp/anaconda3/envs/tf/lib/python3.7/s

In [2]:
def image_loader(image_path, size):
  """ Load Image from Path """
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, (size), cv2.INTER_CUBIC)
  return image

In [3]:
def dataset_preprocessing(dataset_path, labels_file_path, size, image_path_pickle):
  """ Load Images and labels from folder """
  
  with open(labels_file_path, 'r') as f:
    classes = f.read().split('\n')[:-1]

  images = []
  labels = []
  image_paths = []

  for image_name in os.listdir(dataset_path):
    try:
      image_path = os.path.join(dataset_path, image_name)
      images.append(image_loader(image_path, size))
      image_paths.append(image_path)
      for i in range(len(classes)):
        if classes[i] in image_name:
          labels.append(i)
    except:
      pass

  with open(image_path_pickle + ".pickle", 'wb') as f:
    pickle.dump(image_paths, f)

  assert len(images) == len(labels)
  return np.array(images), np.array(labels)

In [4]:
cd = os.getcwd()
dataset_path = cd + '/train/'
labels_file_path = cd + '/labels.txt'
image_path_pickle = cd + '/training_images_pickle'
images, labels = dataset_preprocessing(dataset_path, labels_file_path, (32, 32), image_path_pickle)

In [5]:
images.shape

(50000, 32, 32, 3)

In [6]:
def cosine_distance(training_set_vectors, query_vector, top_n = 50):
    """ Calculate cosine distance between query iamge (vectory) and all training set images (vectors) """
    
    distances = []
    for i in range(len(training_set_vectors)):
        distances.append(cosine(training_set_vector[i], query_vector[0]))
        
    return np.argsort(distances)[:top_n]

In [7]:
def hamming_distance(training_set_vector, query_vector, top_n=50):
    """ Calculate hamming distance between query image (vector) and all training set images (vectors) """
    distances = []
    for i in range(len(training_set_vector)):
        distances.append(hamming(training_set_vector[i], query_vector[0]))
        
    return np.argsort(distances)[:top_n]    

In [8]:
def sparse_accuracy(true_labels, predicted_labels):
    """ Calculates accuracy of model based on softmax outputs """
    
    assert len(true_labels) == len(predicted_labels)
    
    correct = 0
    for i in range(len(true_labels)):
        if np.argmax(predicted_labels[i]) == true_labels[i]:
            correct += 1
            
    return correct / len(true_labels)

In [9]:
# Model Utils

In [10]:
def model_inputs(image_size):
    
    # [Batch_size, height, width, channels]
    inputs = tf.placeholder(dtype=tf.float32, shape=[None, image_size[0], image_size[1], 3], name='images')
    
    targets = tf.placeholder(dtype=tf.int32, shape=[None], name='targets')
    
    dropout_rate = tf.placeholder(dtype=tf.float32, name='dropout_rate')
    
    return inputs, targets, dropout_rate

In [11]:
def conv_block(inputs, number_filt, kernel_size, strides = (1,1), padding = 'SAME', activation = tf.nn.relu, max_pool = True, batch_norm = True):
    """ Defines a conv layer """
    
    conv_features = layer = tf.layers.conv2d(inputs = inputs, 
                                             filters = number_filt, 
                                             kernel_size = kernel_size, 
                                             strides = strides, 
                                             padding = padding, 
                                             activation = activation)
    
    if max_pool:
        layer = tf.layers.max_pooling2d(layer, 
                                        pool_size = (2,2), 
                                        strides = (2,2), 
                                        padding = 'SAME')
        
    if batch_norm:
        layer = tf.layers.batch_normalization(layer)
        
    return layer, conv_features

In [12]:
def dense_block(inputs, units, activation = tf.nn.relu, dropout_rate = None, batch_norm = True):
    """ Defines dense block layer """
    dense_features = layer = tf.layers.dense(inputs = inputs,
                                              units = units,
                                              activation = activation)
    if dropout_rate is not None:
        layer = tf.layers.dropout(layer, 
                                  rate = dropout_rate)
    if batch_norm:
        layer = tf.layers.batch_normalization(layer)
        
    return layer, dense_features

In [13]:
def opt_loss(logits, targets, learning_rate):
    """ Defines optimizer and loss function """
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = targets, logits = logits))
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
    
    return loss, optimizer

In [14]:
# Create Model

In [15]:
class ImageSearchModel(object):
    
    def __init__(self, learning_rate, image_size, number_of_classes = 10):
        tf.reset_default_graph()
        self.inputs, self.targets, self.dropout_rate = model_inputs(image_size)
        
        normalized_image = tf.layers.batch_normalization(self.inputs)
        
        # Conv1
        conv_block_1, self.conv_1_features = conv_block(inputs = normalized_image, 
                                                        number_filt = 64, 
                                                        kernel_size = (3,3), 
                                                        strides = (1,1), 
                                                        padding = 'SAME',
                                                        activation = tf.nn.relu,
                                                        max_pool = True,
                                                        batch_norm = True)
        
        # Conv2
        conv_block_2, self.conv_2_features = conv_block(inputs = conv_block_1, 
                                                        number_filt = 128, 
                                                        kernel_size = (3,3), 
                                                        strides = (1,1), 
                                                        padding = 'SAME',
                                                        activation = tf.nn.relu,
                                                        max_pool = True,
                                                        batch_norm = True)
        
        # Conv3
        conv_block_3, self.conv_3_features = conv_block(inputs = conv_block_2, 
                                                        number_filt = 256, 
                                                        kernel_size = (5,5), 
                                                        strides = (1,1), 
                                                        padding = 'SAME',
                                                        activation = tf.nn.relu,
                                                        max_pool = True,
                                                        batch_norm = True)
        
        # Conv 4
        conv_block_4, self.conv_4_features = conv_block(inputs = conv_block_3, 
                                                        number_filt = 512, 
                                                        kernel_size = (5,5), 
                                                        strides = (1,1), 
                                                        padding = 'SAME',
                                                        activation = tf.nn.relu,
                                                        max_pool = True,
                                                        batch_norm = True)
        
        # Flatten
        flat_layer = tf.layers.flatten(conv_block_4)
        
        # Dense 1
        dense_block_1, self.dense_1_features = dense_block(flat_layer,
                                                      units = 128, 
                                                      activation = tf.nn.relu,
                                                      dropout_rate = self.dropout_rate,
                                                      batch_norm = True)
        
        # Dense 2
        dense_block_2, self.dense_2_features = dense_block(dense_block_1,
                                                      units = 256, 
                                                      activation = tf.nn.relu,
                                                      dropout_rate = self.dropout_rate,
                                                      batch_norm = True)
        
        # Dense 3
        dense_block_3, self.dense_3_features = dense_block(dense_block_2,
                                                      units = 512, 
                                                      activation = tf.nn.relu,
                                                      dropout_rate = self.dropout_rate,
                                                      batch_norm = True)
        
        # Dense 4
        dense_block_4, self.dense_4_features = dense_block(dense_block_3,
                                                      units = 1024, 
                                                      activation = tf.nn.relu,
                                                      dropout_rate = self.dropout_rate,
                                                      batch_norm = True)
        
        logits = tf.layers.dense(inputs = dense_block_4, 
                                 units = number_of_classes, 
                                 activation = None)
        
        self.prediction = tf.nn.softmax(logits)
        
        self.loss, self.opt = opt_loss(logits, 
                                       targets = self.targets, 
                                       learning_rate = learning_rate)

In [16]:
model = ImageSearchModel(0.001, (32, 32), 10)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use keras.layers.dropout instead.


In [17]:
def train(model, epochs, drop_rate, batch_size, data, save_dir, saver_delta = 0.15):
    """ Training Function """
    
    X_train, y_train, X_test, y_test = data
    
    # Session
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    
    # Saver
    saver = tf.train.Saver()
    
    best_test_accuracy = 0
    
    # Training Loop
    for epoch in range(epochs):
        train_accuracy = []
        train_loss = []
        for i in tqdm_notebook(range(len(X_train) // batch_size)):
            start_id = i * batch_size
            end_id = start_id + batch_size
            
            X_batch = X_train[start_id:end_id]
            y_batch = y_train[start_id:end_id]
            
            feed_dict = {model.inputs:X_batch, 
                         model.targets:y_batch, 
                         model.dropout_rate:drop_rate}
            
            _, t_loss, preds_t = session.run([model.opt, model.loss, model.prediction], feed_dict=feed_dict)
            
            train_accuracy.append(sparse_accuracy(y_batch, preds_t))
            train_loss.append(t_loss)
            
        print("Epoch: {}/{}".format(epoch, epochs),
              " | Training accuracy: {}".format(np.mean(train_accuracy)),
              " | Training loss: {}".format(np.mean(train_loss)))
        
        test_accuracy = []
        for i in tqdm_notebook(range(len(X_test) // batch_size)):
            start_id = i * batch_size
            end_id = start_id + batch_size
            
            X_batch = X_test[start_id:end_id]
            y_batch = y_test[start_id:end_id]
            
            feed_dict = {model.inputs:X_batch, 
                         model.dropout_rate:0.0}
            
            preds_test = session.run(model.prediction, feed_dict=feed_dict)
            
            test_accuracy.append(sparse_accuracy(y_batch, preds_test))
            
        print("Test accuracy: {}".format(np.mean(test_accuracy)))
        
        # Save Model
        if np.mean(train_accuracy) > np.mean(test_accuracy):
            if np.abs(np.mean(train_accuracy) - np.mean(test_accuracy)) <= saver_delta:
                if np.mean(test_accuracy) >= best_test_accuracy:
                    best_test_accuracy = np.mean(test_accuracy)
                    saver.save(session, "{}/model_epoch_{}.ckpt".format(save_dir, epoch))
                    
    session.close()

In [18]:
# Hyperparameters

In [19]:
epochs = 20
batch_size = 128
learning_rate = 0.001
dropout_rate = 0.6
image_size = (32, 32)

In [20]:
X_train, y_train = dataset_preprocessing(dataset_path, labels_file_path, image_size, image_path_pickle)

In [21]:
X_train.shape

(50000, 32, 32, 3)

In [22]:
dataset_path_test = cd + '/test/'
image_path_pickle_test = cd + '/test_images_pickle'
X_test, y_test = dataset_preprocessing(dataset_path_test, labels_file_path, image_size, image_path_pickle_test)

In [23]:
X_test.shape

(10000, 32, 32, 3)

In [24]:
# Define Model
model = ImageSearchModel(learning_rate, image_size)

In [25]:
data = (X_train, y_train, X_test, y_test)

In [26]:
#saver_folder = cd + '/cifar/saver/'
#train(model, epochs, dropout_rate, batch_size, data, saver_folder)

In [27]:
# Create Training Set Vector

In [28]:
def create_training_set_vectors(model, X_train, y_train, batch_size, checkpoint_path, image_size, distance='hamming'):
    """ Create training set vectors and save them in a pickle file """
    
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    
    saver = tf.train.Saver()
    saver.restore(session, checkpoint_path)
    
    dense_2_features = []
    dense_4_features = []
    
    for i in tqdm_notebook(range(len(X_train)//batch_size)):
        start_id = i * batch_size
        end_id = start_id + batch_size
        
        X_batch = X_train[start_id:end_id]
        
        feed_dict = {model.inputs:X_batch, model.dropout_rate:0.0}
        
        dense_2, dense_4 = session.run([model.dense_2_features, model.dense_4_features], feed_dict=feed_dict)
        
        dense_2_features.append(dense_2)
        dense_4_features.append(dense_4)
        
    dense_2_features = np.vstack(dense_2_features)
    dense_4_features = np.vstack(dense_4_features)
    
    if distance == 'hamming':
        dense_2_features = np.where(dense_2_features < 0.5, 0, 1)
        dense_4_features = np.where(dense_4_features < 0.5, 0, 1)
        
        training_vectors = np.hstack((dense_2_features, dense_4_features))
        
        with open('hamming_training_vectors_pickle', 'wb') as f:
            pickle.dump(training_vectors, f)
            
    elif distance == 'cosine':
        dense_2_features = np.where(dense_2_features < 0.5, 0, 1)
        dense_4_features = np.where(dense_4_features < 0.5, 0, 1)
        
        training_vectors = np.hstack((dense_2_features, dense_4_features))
        
        with open('cosine_training_vectors_pickle', 'wb') as f:
            pickle.dump(training_vectors, f)

In [29]:
model = ImageSearchModel(learning_rate, image_size)

In [30]:
#checkpoint = cd + '/saver/model_epoch_5.ckpt'
#create_training_set_vectors(model, X_train, y_train, batch_size, checkpoint, image_size, distance='hamming')

In [31]:
def simple_inference(model, session, training_set_vectors, uploaded_image_path, image_size, distance = 'hamming'):
    """ Simple inference for a single image """
    
    image = image_loader(uploaded_image_path, image_size)
    
    feed_dict = {model.inputs:[image], model.dropout_rate:0.0}
    dense_2_features, dense_4_features = session.run([model.dense_2_features, model.dense_4_features], feed_dict=feed_dict)
    
    closest_ids = None
    
    if distance == 'hamming':
        
        dense_2_features = np.where(dense_2_features < 0.5, 0, 1)  # Binarization 
        dense_4_features = np.where(dense_4_features < 0.5, 0, 1)
        
        uploaded_image_vector = np.hstack([dense_2_features, dense_4_features])

        closest_ids = hamming_distance(training_set_vectors, uploaded_image_vector)
        
    elif distnace == 'cosine':
        
        uploaded_image_vector = np.hstack([dense_2_features, dense_4_features])
        closest_ids = cosine_distance(training_set_vectors, uploaded_image_vector)
        
    return closest_ids

In [32]:
# Pipeline Test

In [33]:
model = ImageSearchModel(learning_rate = learning_rate, image_size = image_size)
session = tf.Session()
session.run(tf.global_variables_initializer())

In [34]:
checkpoint = cd + '/saver/model_epoch_5.ckpt'
saver = tf.train.Saver()
saver.restore(session, checkpoint)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /Users/steveyyp/Desktop/cifar/saver/model_epoch_5.ckpt


In [35]:
path = cd + '/training_images_pickle.pickle'
with open(path, 'rb') as f:
    train_image_paths = pickle.load(f)

In [36]:
train_image_paths[:10]

['/Users/steveyyp/Desktop/cifar/train/32270_deer.png',
 '/Users/steveyyp/Desktop/cifar/train/21851_cat.png',
 '/Users/steveyyp/Desktop/cifar/train/48309_deer.png',
 '/Users/steveyyp/Desktop/cifar/train/33547_truck.png',
 '/Users/steveyyp/Desktop/cifar/train/45202_automobile.png',
 '/Users/steveyyp/Desktop/cifar/train/2789_bird.png',
 '/Users/steveyyp/Desktop/cifar/train/24517_horse.png',
 '/Users/steveyyp/Desktop/cifar/train/15193_frog.png',
 '/Users/steveyyp/Desktop/cifar/train/10817_ship.png',
 '/Users/steveyyp/Desktop/cifar/train/49897_ship.png']

In [37]:
# Load train set vectors
path = cd + '/hamming_training_vectors_pickle'
with open(path, 'rb') as f:
    train_set_vectors = pickle.load(f)

In [38]:
test_image = cd + '/test/1052_airplane.png'

In [39]:
results_ids = simple_inference(model, session, train_set_vectors, test_image, image_size, distance='hamming')

In [ ]:
img = image_loader(test_image, image_size)
plt.imshow(img)

In [1]:
# Show Results
w = 10
h = 10
fig = plt.figure(figsize=(16,10))
columns = 10
rows = 5
for i in range(1, columns*rows+1):
    try:
        image = image_loader(train_image_paths[results_ids[i]], image_size)
        fig.add_subplot(rows, columns, i)
        plt.imshow(images)
    except:
        pass
plt.show()

NameError: name 'plt' is not defined